In [1]:
from google import genai
from google.genai import types
import os,re,json
import pathlib
import httpx
import pandas as pd
from io import StringIO
from dotenv import load_dotenv
load_dotenv()

True

In [12]:
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

In [14]:
PROMPT_RAM = """
**Objective:** Extract data from Risk Assessment Matrix (RAM) tables within a provided document and output it into a structured CSV file.

**Phase 1: Identification of Relevant Risk Assessment Matrix Tables**
Carefully examine the document to identify the correct table(s) based on the following ordered criteria:

1.  **Primary Keyword Search:**
    * Look for titles or headings immediately preceding a table that contain any of these exact phrases (case-insensitive): "RAM", "Risk Assessment Matrix", "Overall Level of Concern".
    * Look for these same phrases within the table itself, either as a table heading or as a column header.
2.  **Handling Multi-Page Tables:**
    * If a table matching the criteria spans multiple pages, ensure all segments of the table are consolidated and treated as a single entity.
3.  **Appendix Search (Secondary Location):**
    * If no table is identified in the main body of the document using the primary keywords, search within any section labeled as "Appendix" (or similar, e.g., "Annex").
    * In the appendix, look for tables that contain column names such as (or similar to): "Source", "Risk", "Threat", "Likelihood", AND "Impact". The presence of columns related to both likelihood and impact is crucial here.
4.  **No Table Found:**
    * If, after following all the above steps, no suitable table can be confidently identified, return nothing or an empty CSV.

**Phase 2: Data Extraction and CSV Output Formatting**
Once a relevant table is identified, extract its data and structure it into a CSV file with the following precise columns. Apply these rules to each row of the identified table(s):

1.  **`Heading`**:
    * Content: Store the table's primary title or caption if available. If multiple (e.g., a number and a title), combine them. If no explicit title/caption is found directly associated with the table, leave this field empty for that table's rows.
2.  **`Risk Type`**:
    * Source: Identify the row, column or section headers  in the source table that most likely contains types of risks. They oftern appear as 'Global Risks','External Risks' or 'Domestic Risks','Country Specific Risks', or similar.
    * Content: it should be classificed as either "Global Risks" Or "Domestic Risks". This field cannot be empty. 
3.  **`Risk`**:
    * Source: Identify the column in the source table that most likely contains risk descriptions. Look for column headers containing keywords like "Risk", "Threat", "Hazard", or "Source".
    * Content: Extract the full text from this cell. This field cannot be empty. It often contains long, descriptive text.
4.  **`Time Horizon`**:
    * Source: Identify the column in the source table that most likely contains description on the time horizon of the risk. Look for column headers containing keywords like "Time Horizon".
    * Content: Extract coresponding time horizon description from this cell. There are cases where time horizon information is mix together with likelihood infomation, Make sure only extract time horizon infomation. This field is optional, if the table does not contain this information, leave it empty.
5.  **`Likelihood Full`**:
    * Source: Identify the column in the source table that most likely contains detailed likelihood information. Look for column headers containing the keyword "Likelihood".
    * Content: Extract EVERYTHING from this cell, including multiple paragraphs, bullet points, and any text separated by large blank spaces. This field cannot be empty.
6.  **`Likelihood`**:
    * Source: From the content extracted for `Likelihood Full`.
    * Content: Extract only the first line or primary categorical value that typically describes the level of likelihood (e.g., "High", "Medium", "Low", "Likely", "Unlikely", "Remote", "Possible", "Frequent"). If the first line is a detailed sentence, try to infer the single-word or short-phrase category.
7.  **`Impact Full`**:
    * Source: Identify the column in the source table that most likely contains detailed impact information. Look for column headers containing the keyword "Impact" or "Consequence".
    * Content: Extract EVERYTHING from this cell, including multiple paragraphs, bullet points, and any text separated by large blank spaces. This field cannot be empty.
8.  **`Impact`**:
    * Source: From the content extracted for `Impact Full`.
    * Content: Extract only the first line or primary categorical value that typically describes the level of impact (e.g., "High", "Medium", "Low", "Severe", "Moderate", "Minor", "Catastrophic", "Significant"). If the first line is a detailed sentence, try to infer the single-word or short-phrase category.
9.  **`Policy Response`**:
    * Source: Identify the column in the source table that most likely contains information about policy responses to the risk. Look for column headers containing keywords like "Policy", "Response", "Mitigation", "Action", or "Measure".
    * Content: Extract the full text from this cell. If no such column is found, or if the cell for a given risk is empty, leave this field empty in the CSV.

**Phase 3: Important Processing Details**

1.  **Footnote Removal:** If footnotes or source notes are present directly beneath the table content (and are clearly distinguishable from the main table data), they should be excluded from the extraction.
2.  **Data Integrity:**
    * If a row within an identified RAM table is missing data for the determined "Risk", "Likelihood Full", or "Impact Full" source columns, still include the row but leave the corresponding CSV cells empty for that missing piece (this slightly overrides "can't be empty" for cases where a source cell *within an otherwise valid RAM row* is blank). The expectation is that most rows in a valid RAM will have this data.
    * Prioritize accurate mapping of table columns to the requested CSV columns.

**Output:**
Provide the extracted data as a single CSV file.

"""

In [16]:
def extract_csv_content(text):
    match = re.search(r"```csv(.*?)```", text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return text.strip()

def clean_markdown_fence(text):
    """
    Removes Markdown code fences like ```csv ... ``` and returns only the inner content.
    Handles cases with or without newlines, with or without language tags.
    """
    # Match fenced code block with or without language tag
    match = re.search(r"```(?:\w+)?(.*?)```", text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return text.strip()

def clean_RAM(ram):

    letter_pattern = r'[a-zA-Z]'
    keyword_pattern = r'(risk|assessment|matrix|ram)'
    likelihood_impact_keywords = ['high', 'medium', 'low', 'moderate', 'severe']

    # Remove rows where 'Likelihood' or 'Impact' columns contain no letters

    cleaned_df = ram[
        ram['Likelihood'].str.contains(letter_pattern, na=False) &
        ram['Impact'].str.contains(letter_pattern, na=False) &
        ram['Likelihood Full'].str.contains(letter_pattern, na=False) &
        ram['Impact Full'].str.contains(letter_pattern, na=False)
    ]

    # Remove rows where 'Heading' does not contain RAM keywords if 'Heading' is not missing
    cleaned_df = cleaned_df[
        cleaned_df['Heading'].isna() | 
        (cleaned_df['Heading'].str.count(keyword_pattern, flags=re.IGNORECASE) >=2)
    ]

    # Remove rows where 'Likelihood' or 'Impact' columns do not contain keywords from high, medium, low
    cleaned_df = cleaned_df[
        cleaned_df['Likelihood'].str.contains('|'.join(likelihood_impact_keywords), case=False, na=False) |
        cleaned_df['Impact'].str.contains('|'.join(likelihood_impact_keywords), case=False, na=False)
    ]

    # Keep only the specified columns
    cleaned_df = cleaned_df[['Heading', 'Risk Type','Risk', 'Time Horizon','Likelihood', 'Likelihood Full', 'Impact', 'Impact Full','Policy Response']]

    print(f"Rows before cleaning: {len(ram)}")
    print(f"Rows after cleaning: {len(cleaned_df)}")

    return cleaned_df

- process to convert to csv directly 

In [35]:
# Retrieve and encode the PDF byte
f_path= "/ephemeral/home/xiong/data/Fund/RAM_Table/RAM_Table_Examples/536_2023_0.pdf"
#### experimented with various coner cases 124_2022_0; 466_2022_0;  536_2023_0
filepath = pathlib.Path(f_path)
filepath.exists()

True

In [36]:
response = client.models.generate_content(
          model="gemini-2.5-pro-preview-05-06",
          contents=[
              types.Part.from_bytes(
                data=filepath.read_bytes(),
                mime_type='application/pdf',
              ),
              PROMPT_RAM])


In [37]:
df = pd.read_csv(StringIO(extract_csv_content(response.text)))
if df.empty:
    print("No relevant RAM table found.")
else:
    df = clean_RAM(df)

Rows before cleaning: 7
Rows after cleaning: 7


In [38]:
df

,Heading,Risk Type,Risk,Time Horizon,Likelihood,Likelihood Full,Impact,Impact Full,Policy Response
0,Annex II. Risk Assessment Matrix¹,Global Risks,Abrupt global slowdown or recession 1/\n• EMDE...,NaN,Medium,Medium,High,"High. Lower GDP growth, due to weaker investme...",Use available fiscal space to provide targeted...
1,Annex II. Risk Assessment Matrix¹,Global Risks,Systemic financial instability 2/,NaN,Medium,Medium,Medium,Medium. An increase in capital outflows that l...,Provided the monetary policy stance remains ad...
2,Annex II. Risk Assessment Matrix¹,Global Risks,Commodity price volatility 3/,NaN,Medium,Medium,Medium,Medium. The economic impacts will critically d...,Use fiscal and monetary policy countercyclical...
3,Annex II. Risk Assessment Matrix¹,Global Risks,Deepening geo-economic fragmentation 4/,NaN,High,High,Medium,Medium. Weaker exports; reduced FDI inflows; i...,Accelerate structural reforms to facilitate th...
4,Annex II. Risk Assessment Matrix¹,Domestic Risks,"Persistently high domestic inflation, due to g...",NaN,Medium,Medium,High,High. Slower closing of the output gap amid we...,Use available fiscal space to provide targeted...
5,Annex II. Risk Assessment Matrix¹,Domestic Risks,Natural disasters\nMajor natural disasters (e....,NaN,Medium,Medium,Low,Low. Disruption in economic activity in the af...,Prioritize expenditures to affected households...
6,Annex II. Risk Assessment Matrix¹,Domestic Risks,"A stronger boost from pent-up demand, together...",NaN,Medium,Medium,Medium,Medium. Stronger private sector investment as ...,Monetary policy can be tightened if inflation ...


- process to convert to markdown 

In [39]:
from IPython.display import Markdown, display

In [40]:
PROMPT_RAM_MD = """
**Objective:** Convert data from Risk Assessment Matrix (RAM) tables within a provided document and output it into a Markdown.

**Phase 1: Identification of Relevant Risk Assessment Matrix Tables**
Carefully examine the document to identify the correct table(s) based on the following ordered criteria:

1.  **Primary Keyword Search:**
    * Look for titles or headings immediately preceding a table that contain any of these exact phrases (case-insensitive): "RAM", "Risk Assessment Matrix", "Overall Level of Concern".
    * Look for these same phrases within the table itself, either as a table heading or as a column header.
2.  **Handling Multi-Page Tables:**
    * If a table matching the criteria spans multiple pages, ensure all segments of the table are consolidated and treated as a single entity.
3.  **Appendix Search (Secondary Location):**
    * If no table is identified in the main body of the document using the primary keywords, search within any section labeled as "Appendix" (or similar, e.g., "Annex").
    * In the appendix, look for tables that contain column names such as (or similar to): "Source", "Risk", "Threat", "Likelihood", AND "Impact". The presence of columns related to both likelihood and impact is crucial here.
4.  **No Table Found:**
    * If, after following all the above steps, no suitable table can be confidently identified, return nothing.
    
**Phase 2: Data Extraction and Markdown Output Formatting**
1.  Once a relevant table is identified, extract its data and structure it into a Markdown format.

**Phase 3: Important Processing Details**
1.  **Footnote Removal:** If footnotes or source notes are present directly beneath the table content (and are clearly distinguishable from the main table data), they should be excluded from the extraction.
2.  **Data Integrity:** the output content should be the exact content from the table, without any paraphrasing, summarization or modification in wording. 

**Output:**
Provide the extracted data as a Markdown file.
"""

In [41]:
response = client.models.generate_content(
          model="gemini-2.5-pro-preview-05-06",
          contents=[
              types.Part.from_bytes(
                data=filepath.read_bytes(),
                mime_type='application/pdf',
              ),
              PROMPT_RAM_MD])

In [42]:
# print(response.text)

In [43]:
display(Markdown(clean_markdown_fence(response.text)))

# Annex II. Risk Assessment Matrix¹

## Global

| Source of Risks | Likelihood | Expected Impact | Policy Recommendation |
|---|---|---|---|
| **Abrupt global slowdown or recession 1/** <br> • **EMDES:** A new bout of global financial tightening, possibly combined with volatile commodity prices, leads to spiking risk premia, debt distress, widening of external imbalances, fiscal pressures, and sudden stops. | Medium | High. Lower GDP growth, due to weaker investment and exports; a decline in capital inflows, leading to currency depreciation and tighter domestic credit conditions; higher poverty rate. | Use available fiscal space to provide targeted support, while allowing the exchange rate to act as a shock absorber. FXI could be used to address disorderly market conditions (e.g., a sharp pickup in the UIP premium), easing the burden on monetary policy. |
| **Systemic financial instability 2/** | Medium | Medium. An increase in capital outflows that lead to large exchange rate depreciation and volatility; tighter financial conditions; and upward pressures on inflation through exchange rate passthrough. | Provided the monetary policy stance remains adequate to address inflation risks, and considering Indonesia's shallow FX markets, the use of FXI may be appropriate under certain circumstances (see Annex V for further discussions). |
| **Commodity price volatility 3/** | Medium | Medium. The economic impacts will critically depend on whether commodity prices rise or decline, given Indonesia's commodity exporter status; A sharper-than-expected decline in international commodity prices would lead to lower growth, worsen the external balance, and lead to a faster moderation in inflation momentum. | Use fiscal and monetary policy countercyclically to stabilize output and inflation. Seek to make progress on reforming energy subsidies. The exchange rate should remain flexible and determined by market forces. |
| **Deepening geo-economic fragmentation 4/** | High | Medium. Weaker exports; reduced FDI inflows; increased uncertainty leading to weaker investment. | Accelerate structural reforms to facilitate the transition to a higher value-added and greener economy, including through additional investment in education and infrastructure and by reducing trade restrictions. |

## Domestic

| Source of Risks | Likelihood | Expected Impact | Policy Recommendation |
|---|---|---|---|
| **Persistently high domestic inflation,** due to global or domestic supply disruptions, coupled with a slower-than-expected growth. | Medium | High. Slower closing of the output gap amid weaker private consumption and investment; a decline in capital inflows due to less positive economic prospects. | Use available fiscal space to provide targeted policy support to low-income households and small businesses. Monetary policy should carefully balance risks to output, inflation and financial stability and be tightened further if inflation surprises on the upside or if global monetary conditions tighten substantially, leading to large depreciation pressures on the rupiah. |
| **Natural disasters** <br> Major natural disasters (e.g., volcanic eruptions and/or earthquakes) disrupt economic activity and affect sentiment, resulting in higher fiscal expenditures. | Medium | Low. Disruption in economic activity in the affected region; slower economic growth accompanied by a decline in portfolio inflows. | Prioritize expenditures to affected households and businesses. If the economy slows significantly, loosen monetary and fiscal policies to accelerate the recovery, strengthen monitoring of corporate vulnerabilities. |
| **A stronger boost from pent-up demand,** together with a faster recovery in China, a faster fall in inflation domestically or in Indonesia's key trading partners can lift economic prospects | Medium | Medium. Stronger private sector investment as economic prospects improve, contributing to a faster-than-projected closing of the output gap; inflation stays high for longer; and the recovery in bank credit accelerates. | Monetary policy can be tightened if inflation momentum picks up. Macroprudential policies should be tightened if credit growth accelerates, closing the credit gap earlier. |

### Experiment with full document without specific Table Identification 

In [44]:
full_doc_path= '/ephemeral/home/xiong/data/Fund/pdf_parse/Fund_Document/input/All_AIV_2008-2025_PDF/536_2023_0.pdf'
filepath = pathlib.Path(f_path)
filepath.exists()

True

In [45]:
response = client.models.generate_content(
          model="gemini-2.5-pro-preview-05-06",
          contents=[
              types.Part.from_bytes(
                data=filepath.read_bytes(),
                mime_type='application/pdf',
              ),
              PROMPT_RAM])

In [46]:
df = pd.read_csv(StringIO(extract_csv_content(response.text)))
if df.empty:
    print("No relevant RAM table found.")
else:
    df = clean_RAM(df)

Rows before cleaning: 7
Rows after cleaning: 7


In [50]:
df

,Heading,Risk Type,Risk,Time Horizon,Likelihood,Likelihood Full,Impact,Impact Full,Policy Response
0,Annex II. Risk Assessment Matrix¹,Global Risks,• EMDES: A new bout of global financial tighte...,NaN,Medium,Medium,High,"High. Lower GDP growth, due to weaker investme...",Use available fiscal space to provide targeted...
1,Annex II. Risk Assessment Matrix¹,Global Risks,Systemic financial instability 2/,NaN,Medium,Medium,Medium,Medium. An increase in capital outflows that l...,Provided the monetary policy stance remains ad...
2,Annex II. Risk Assessment Matrix¹,Global Risks,Commodity price volatility 3/,NaN,Medium,Medium,Medium,Medium. The economic impacts will critically d...,Use fiscal and monetary policy countercyclical...
3,Annex II. Risk Assessment Matrix¹,Global Risks,Deepening geo-economic fragmentation 4/,NaN,High,High,Medium,Medium. Weaker exports; reduced FDI inflows; i...,Accelerate structural reforms to facilitate th...
4,Annex II. Risk Assessment Matrix¹,Domestic Risks,"Persistently high domestic inflation, due to g...",NaN,Medium,Medium,High,High. Slower closing of the output gap amid we...,Use available fiscal space to provide targeted...
5,Annex II. Risk Assessment Matrix¹,Domestic Risks,Natural disasters Major natural disasters (e.g...,NaN,Medium,Medium,Low,Low. Disruption in economic activity in the af...,Prioritize expenditures to affected households...
6,Annex II. Risk Assessment Matrix¹,Domestic Risks,"A stronger boost from pent-up demand, together...",NaN,Medium,Medium,Medium,Medium. Stronger private sector investment as ...,Monetary policy can be tightened if inflation ...


In [48]:
response = client.models.generate_content(
          model="gemini-2.5-pro-preview-05-06",
          contents=[
              types.Part.from_bytes(
                data=filepath.read_bytes(),
                mime_type='application/pdf',
              ),
              PROMPT_RAM_MD])
display(Markdown(clean_markdown_fence(response.text)))

# Annex II. Risk Assessment Matrix¹

| Category | Source of Risks | Likelihoo | Expected Impact | Policy Recommendation |
|---|---|---|---|---|
| **Global** | **Abrupt global slowdown or recession 1/** <br> • EMDES: A new bout of global financial tightening, possibly combined with volatile commodity prices, leads to spiking risk premia, debt distress, widening of external imbalances, fiscal pressures, and sudden stops. | Medium | **High.** Lower GDP growth, due to weaker investment and exports; a decline in capital inflows, leading to currency depreciation and tighter domestic credit conditions; higher poverty rate. | Use available fiscal space to provide targeted support, while allowing the exchange rate to act as a shock absorber. FXI could be used to address disorderly market conditions (e.g., a sharp pickup in the UIP premium), easing the burden on monetary policy. |
| **Global** | **Systemic financial instability 2/** | Medium | **Medium.** An increase in capital outflows that lead to large exchange rate depreciation and volatility; tighter financial conditions; and upward pressures on inflation through exchange rate passthrough. | Provided the monetary policy stance remains adequate to address inflation risks, and considering Indonesia's shallow FX markets, the use of FXI may be appropriate under certain circumstances (see Annex V for further discussions). |
| **Global** | **Commodity price volatility 3/** | Medium | **Medium.** The economic impacts will critically depend on whether commodity prices rise or decline, given Indonesia's commodity exporter status; A sharper-than-expected decline in international commodity prices would lead to lower growth, worsen the external balance, and lead to a faster moderation in inflation momentum. | Use fiscal and monetary policy countercyclically to stabilize output and inflation. Seek to make progress on reforming energy subsidies. The exchange rate should remain flexible and determined by market forces. |
| **Global** | **Deepening geo-economic fragmentation 4/** | High | **Medium.** Weaker exports; reduced FDI inflows; increased uncertainty leading to weaker investment. | Accelerate structural reforms to facilitate the transition to a higher value-added and greener economy, including through additional investment in education and infrastructure and by reducing trade restrictions. |
| **Domestic** | **Persistently high domestic inflation,** due to global or domestic supply disruptions, coupled with a slower-than-expected growth. | Medium | **High.** Slower closing of the output gap amid weaker private consumption and investment; a decline in capital inflows due to less positive economic prospects. | Use available fiscal space to provide targeted policy support to low-income households and small businesses. Monetary policy should carefully balance risks to output, inflation and financial stability and be tightened further if inflation surprises on the upside or if global monetary conditions tighten substantially, leading to large depreciation pressures on the rupiah. |
| **Domestic** | **Natural disasters** <br> Major natural disasters (e.g., volcanic eruptions and/or earthquakes) disrupt economic activity and affect sentiment, resulting in higher fiscal expenditures. | Medium | **Low.** Disruption in economic activity in the affected region; slower economic growth accompanied by a decline in portfolio inflows. | Prioritize expenditures to affected households and businesses. If the economy slows significantly, loosen monetary and fiscal policies to accelerate the recovery, strengthen monitoring of corporate vulnerabilities. |
| **Domestic** | **A stronger boost from pent-up demand,** together with a faster recovery in China, a faster fall in inflation domestically or in Indonesia's key trading partners can lift economic prospects | Medium | **Medium.** Stronger private sector investment as economic prospects improve, contributing to a faster-than-projected closing of the output gap; inflation stays high for longer; and the recovery in bank credit accelerates. | Monetary policy can be tightened if inflation momentum picks up. Macroprudential policies should be tightened if credit growth accelerates, closing the credit gap earlier. |

### process with json output 

In [99]:
# GenAI configuration
generation_config = {
    "temperature": 0.01,
    "top_p": 0.95,
    "top_k": 40,
    "max_output_tokens": 8000, # put it as 8k
    "response_mime_type": "application/json",
    }

In [100]:
response = client.models.generate_content(
          model="gemini-2.5-pro-preview-05-06",
          config=generation_config,
          contents=[
              types.Part.from_bytes(
                data=filepath.read_bytes(),
                mime_type='application/pdf',
              ),
              PROMPT_RAM])

In [108]:
response_data = json.loads(response.text)
response_data


[{'Heading': 'Annex IV. Risk Assessment Matrix',
  'Risk Type': 'External Risks',
  'Risk': "Russia's invasion of Ukraine leads to escalation of sanctions and other disruptions. Russia is disconnected almost completely from the global financial system and large parts of the trading system. This, combined with Russian countersanctions and secondary sanctions on countries and companies that continue business with Russia, leads to even higher commodity prices, refugee migration, tighter financial conditions.",
  'Likelihood Full': 'High',
  'Likelihood': 'High',
  'Impact Full': 'High\n• Lower consumption and production, particular manufacturing industry due to gas supply disruption.\n• Lower exports due to weakened external demand.\n• Lower investment due to heightened uncertainty and tightening of financial conditions.\n• Increase systemic risks in the financial sector.',
  'Impact': 'High',
  'Policy Response': 'Accelerate green transitions\nDiversify energy sources\nProvide targeted f

In [107]:
df = pd.DataFrame(response_data)
df

,Heading,Risk Type,Risk,Likelihood Full,Likelihood,Impact Full,Impact,Policy Response
0,Annex IV. Risk Assessment Matrix,External Risks,Russia's invasion of Ukraine leads to escalati...,High,High,"High\n• Lower consumption and production, part...",High,Accelerate green transitions\nDiversify energy...
1,Annex IV. Risk Assessment Matrix,External Risks,Rising and volatile food and energy prices. Co...,High,High,Medium\n• Lower growth due to weak consumption...,Medium,Provide targeted fiscal support to vulnerable ...
2,Annex IV. Risk Assessment Matrix,External Risks,Outbreaks of lethal and highly contagious COVI...,High,High,"Medium\n• Tourist arrivals remain low, lowerin...",Medium,Targeted support to direct-affected businesses...
3,Annex IV. Risk Assessment Matrix,External Risks,De-anchoring of inflation expectations in the ...,Medium\nfor the US\nand\nMedium-\nlow for the\...,Medium,Medium\n• Weaker confidence could reduce inves...,Medium,• Implement reforms to boost productivity and ...
4,Annex IV. Risk Assessment Matrix,External Risks,Geopolitical tensions and de-globalization. In...,High,High,High\n• Tariff and non-tariff restrictions fur...,High,• Allow automatic stabilizer fully to operate....
5,Annex IV. Risk Assessment Matrix,External Risks,Higher frequency and severity of natural disas...,Medium,Medium,Medium\n• Weaker demand for winter tourism and...,Medium,• Support affected sectors and rebuild damaged...
6,Annex IV. Risk Assessment Matrix,Domestic Risks,Extended global supply chain disruptions. Pers...,High,High,Medium/High\n• Persistently high inflation\n• ...,Medium/High,Provide targeted fiscal support to vulnerable ...
7,Annex IV. Risk Assessment Matrix,Domestic Risks,Weakness in coordinating and implementing dome...,Medium,Medium,Medium\n• Reform setback could undermine confi...,Medium,Outline reform details in a comprehensive pack...
8,Annex IV. Risk Assessment Matrix,Domestic Risks,Contagion from CESEE. The stock of foreign cur...,High,High,Medium\n• Depreciation in host country could r...,Medium,Boost structural profitability through reducin...
